<a href="https://www.kaggle.com/code/ngocthuy/ev-analysis?scriptVersionId=196304866" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from urllib.request import urlopen
import json
import geopandas as gpd

# **1. Data Loading, Data Cleaning**

In [2]:
df = pd.read_csv('/kaggle/input/electric-vehicle-data-washington-state-department/Electric_Vehicle_Population_Data.csv')
df.head(5)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,1C4RJXN66R,Snohomish,Everett,WA,98204.0,2024,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21.0,0.0,21.0,261311557,POINT (-122.2507211 47.8976713),PUGET SOUND ENERGY INC,5.306104e+10
1,KNDJX3AEXG,King,Renton,WA,98058.0,2016,KIA,SOUL,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,93.0,31950.0,11.0,210641315,POINT (-122.1476337 47.4438471),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
2,5YJ3E1EA3L,King,Seattle,WA,98125.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,266.0,0.0,46.0,124517347,POINT (-122.304356 47.715668),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
3,1G1RC6S5XH,Kitsap,Port Orchard,WA,98367.0,2017,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,53.0,0.0,26.0,7832933,POINT (-122.6530052 47.4739066),PUGET SOUND ENERGY INC,5.303509e+10
4,5UXTA6C09P,Snohomish,Monroe,WA,98272.0,2023,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,30.0,0.0,39.0,235249262,POINT (-121.968385 47.854897),PUGET SOUND ENERGY INC,5.306105e+10


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194232 entries, 0 to 194231
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         194232 non-null  object 
 1   County                                             194223 non-null  object 
 2   City                                               194223 non-null  object 
 3   State                                              194232 non-null  object 
 4   Postal Code                                        194223 non-null  float64
 5   Model Year                                         194232 non-null  int64  
 6   Make                                               194232 non-null  object 
 7   Model                                              194232 non-null  object 
 8   Electric Vehicle Type                              194232 non-null  object

In [4]:
df.columns

Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Legislative District', 'DOL Vehicle ID',
       'Vehicle Location', 'Electric Utility', '2020 Census Tract'],
      dtype='object')

In [5]:
drop_cols=['VIN (1-10)','State','Base MSRP', 'Legislative District', 'DOL Vehicle ID',
       'Vehicle Location', 'Electric Utility', '2020 Census Tract']
df = df.drop(columns=drop_cols, axis=1)

In [6]:
df.nunique()

County                                               198
City                                                 756
Postal Code                                          905
Model Year                                            22
Make                                                  42
Model                                                151
Electric Vehicle Type                                  2
Clean Alternative Fuel Vehicle (CAFV) Eligibility      3
Electric Range                                       104
dtype: int64

In [7]:
df = df.dropna(axis=0)

# **2. Data Analysis**

#  2.1 Electric Vehicle Type Analysis

In [8]:
# Prepare the data
df_EV = df.groupby('Electric Vehicle Type').agg(No_of_Vehicles=('Electric Vehicle Type', 'count'))\
            .reset_index().sort_values(by='No_of_Vehicles', ascending=False)
colors = ['#636EFA', '#EF553B']

# Create the bar chart for the distribution of electric vehicle types
fig_distribution = go.Bar(x=df_EV['Electric Vehicle Type'], y=df_EV['No_of_Vehicles'], 
                          marker=dict(color=colors), name='Distribution')

# Create the pie chart for market share by electric vehicle type
fig_proportion = go.Pie(labels=df_EV['Electric Vehicle Type'], values=df_EV['No_of_Vehicles'], 
                        marker=dict(colors=colors), name='Market Share')

# Create the subplot figure
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('Distribution of Electric Vehicle Type', 'Market Share by Electric Vehicle Type'),
                    horizontal_spacing=0.15, specs=[[{'type': 'xy'}, {'type': 'pie'}]])

# Add the bar chart to the first subplot
fig.add_trace(fig_distribution, row=1, col=1)
fig.add_trace(fig_proportion, row=1, col=2)
fig.update_layout(height=600, width=1200, 
                  title_text='Electric Vehicle Type Analysis', title_x=0.4)
fig.show()

# 2.2 Unit Sales by County and City

In [9]:
df_sub = df.groupby(['County', 'City']).agg(No_of_Vehicles=('City', 'count')).reset_index().sort_values(by='No_of_Vehicles', ascending=False)

fig = px.sunburst(df_sub, path=['County','City'], values='No_of_Vehicles', 
                  title='No_of_Vehicles by County and City')
fig.update_layout(height=600, width=1000, title_x=0.5)
fig.show()

In [10]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    geojson_data = json.load(response)

gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])
gdf_wa = gdf[gdf['STATE']=='53']


df_county = df_sub.groupby('County').agg(No_of_Vehicles=('No_of_Vehicles', 'sum')).reset_index().sort_values(by='No_of_Vehicles', ascending=False)
merge_df = gdf_wa.merge(df_county, left_on='NAME', right_on='County')

fig = px.choropleth(merge_df, geojson=merge_df.geometry.__geo_interface__,
                    locations=merge_df.index, color='No_of_Vehicles',
                    color_continuous_scale = 'Reds', range_color = (1, 60000),
                    hover_name='County')

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(height=600, width=800, title_text='Washington State County Map', title_x=0.5)
fig.show()


# 2.3 Top 10 county, City with highest EV

In [11]:
fig_County = go.Bar(x = df_county.head(10)['County'], y = df_county.head(10)['No_of_Vehicles'], 
                    name='Top 10 county with highest Electric Vehicles')

fig_City = go.Bar(x = df_sub.head(10)['City'], y=df_sub.head(10)['No_of_Vehicles'], 
                  name = 'Top 10 City with highest Electric Vehicles')

fig = make_subplots(rows=1, cols=2, subplot_titles=('Distribution of Electric Vehicle Type', 'Market Share by Electric Vehicle Type'),
                   horizontal_spacing=0.15)

fig.add_trace(fig_County, row=1, col=1)
fig.add_trace(fig_City, row=1, col=2)
fig.update_layout(height=600, width=1000, title='No_of_Vehicles by County, City',
                 legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='center', x=0.5),
                 title_x=0.5, margin=dict(t=120))

fig.update_xaxes(tickangle=-45)
fig.show()

# 2.4 Top Make and Model with highest Electric Vehicles

In [12]:
df_Make = df.groupby('Make').agg(No_of_Vehicles=('Make', 'count')).reset_index().sort_values(by='No_of_Vehicles', ascending=False).head(10)

fig_Make = go.Treemap(values = df_Make['No_of_Vehicles'], labels=df_Make['Make'],
                           parents=[''] * len(df_Make), textfont_size=20,root_color="lightblue",
                           marker_colorscale = 'Blues')

df_Model = df.groupby('Model').agg(No_of_Vehicles=('Model', 'count')).reset_index().sort_values(by='No_of_Vehicles', ascending=False).head(10)

fig_Model = go.Scatter(x = df_Model['Model'], y = df_Model['No_of_Vehicles'], mode='markers',
                       marker = dict(size=df_Model['No_of_Vehicles'], color=df_Model['No_of_Vehicles'], colorscale='Reds',
                                    sizemode='area', sizemin=4, sizeref=2.*max(df_Model['No_of_Vehicles'])/(60.**2)),
                       text=df_Model['Model'])

fig = make_subplots(rows=1, cols=2, subplot_titles=('Top 10 Make with highest EV','Top 10 Model with highest EV'),
                   specs=[[{'type': 'treemap'}, {'type': 'xy'}]])

fig.add_trace(fig_Make, row=1, col=1)
fig.add_trace(fig_Model, row=1, col=2)
fig.update_layout(title = 'Top 10 Make , Model with highest Electric Vehicles', 
                  title_x = 0.5, margin = dict(t=100, l=25, r=25, b=25))
fig.show()



# 2.5 Growth of Electric Vehicle based on Model Year

In [13]:
df_ModelYear = df.groupby('Model Year').agg(No_of_Vehicles = ('Make', 'count')).reset_index()
df_BEV = df[df['Electric Vehicle Type']=='Plug-in Hybrid Electric Vehicle (PHEV)']\
        .groupby('Model Year').agg(BEV = ('Make', 'count')).reset_index()
df_PHEV = df[df['Electric Vehicle Type']=='Battery Electric Vehicle (BEV)']\
        .groupby('Model Year').agg(PHEV = ('Make', 'count')).reset_index()

df_ModelYear = df_ModelYear.merge(df_BEV, on='Model Year', how='left').merge(df_PHEV, on='Model Year', how='left')

Since 2024 is not yet over, we will filter out the data from 2010 to 2023

In [14]:
df_ModelYear = df_ModelYear[(df_ModelYear['Model Year'] >= 2010) & (df_ModelYear['Model Year'] <= 2023)]
df_ModelYear

,Model Year,No_of_Vehicles,BEV,PHEV
6,2010,24,3.0,21
7,2011,730,68.0,662
8,2012,1594,855.0,739
9,2013,4396,1590.0,2806
10,2014,3538,1792.0,1746
11,2015,4797,1308.0,3489
12,2016,5547,1762.0,3785
13,2017,8661,4074.0,4587
14,2018,14450,4375.0,10075
15,2019,10904,2030.0,8874


In [15]:
fig_ModelYear = go.Scatter(x=df_ModelYear['Model Year'], y = df_ModelYear['No_of_Vehicles'], name='Total No of EV')
fig_BEV = go.Scatter(x=df_ModelYear['Model Year'], y = df_ModelYear['BEV'], name= 'BEV', line=dict(color='blue'))
fig_PHEV = go.Scatter(x=df_ModelYear['Model Year'], y = df_ModelYear['PHEV'], name='PHEV',line=dict(color='green'))

fig = make_subplots(rows=1, cols = 2,
                    subplot_titles=('Total No of Vehicle over the Years','Total No of Vehicle BEV, PHEV over the Years'))
fig.add_trace(fig_ModelYear, row=1, col=1)
fig.add_trace(fig_BEV, row=1, col=2)
fig.add_trace(fig_PHEV, row=1, col=2)
fig.update_layout(title = 'Number of vehicles over the years from 2012 to 2023', 
                  title_x = 0.5, margin = dict(t=100, l=25, r=25, b=25),
                  xaxis=dict(title='Model Year', dtick=1), yaxis_title='Number of Vehicles',
                  xaxis2=dict(title='Model Year', dtick=1), yaxis2_title='Number of Vehicles')
fig.update_xaxes(tickangle=-45)
fig.show()


# 2.5 Top 3 EV Make Vs Their Years Of Model

In [16]:
df_sub = df[df['Make'].isin(['TESLA', 'CHEVROLET', 'NISSAN'])].groupby(['Make', 'Model Year'])\
        .agg(No_of_Vehicles=('Make', 'count')).reset_index()

df_sub['Make'] = pd.Categorical(df_sub['Make'], categories=['TESLA', 'CHEVROLET', 'NISSAN'], ordered=True)

df_sub = df_sub.sort_values(by=['Make', 'Model Year'])
df_sub = df_sub[(df_sub['Model Year'] >= 2010) & (df_sub['Model Year'] <= 2023)]
df_sub

,Make,Model Year,No_of_Vehicles
30,TESLA,2010,21
31,TESLA,2011,6
32,TESLA,2012,129
33,TESLA,2013,718
34,TESLA,2014,639
35,TESLA,2015,1029
36,TESLA,2016,1619
37,TESLA,2017,1672
38,TESLA,2018,8024
39,TESLA,2019,4620


In [17]:
df_tesla = df_sub[df_sub['Make'] == 'TESLA']
df_chevrolet = df_sub[df_sub['Make'] == 'CHEVROLET']
df_nissan = df_sub[df_sub['Make'] == 'NISSAN']

fig_tesla = go.Scatter(x=df_tesla['Model Year'], y=df_tesla['No_of_Vehicles'],
                       mode='lines+markers', name='TESLA', line=dict(color='red'))

fig_chevrolet = go.Scatter(x=df_chevrolet['Model Year'], y=df_chevrolet['No_of_Vehicles'],
                           mode='lines+markers', name='CHEVROLET', line=dict(color='blue'))

fig_nissan = go.Scatter(x=df_nissan['Model Year'], y=df_nissan['No_of_Vehicles'],
                        mode='lines+markers', name='NISSAN', line=dict(color='green'))

fig = go.Figure()
fig.add_trace(fig_tesla)
fig.add_trace(fig_chevrolet)
fig.add_trace(fig_nissan)


fig.update_layout(
    title='Number of Vehicles Over the Years for Tesla, Chevrolet, and Nissan',
    xaxis_title='Model Year',
    yaxis_title='Number of Vehicles',
    xaxis=dict(dtick=1), 
    yaxis=dict(rangemode='tozero'), 
    title_x=0.5,  
    margin=dict(t=100, l=25, r=25, b=25)
)

fig.show()

# 2.6 Model with the highest number of vehicles each year

In [18]:
df_Model = df.groupby(['Model Year', 'Model']).agg(No_of_Vehicles = ('Model', 'count')).reset_index()
df_Model = df_Model.sort_values(['Model Year', 'No_of_Vehicles'], ascending=[True, False])
df_Model = df_Model.drop_duplicates(subset=['Model Year'], keep='first')
df_Model = df_Model[(df_Model['Model Year'] >= 2010) & (df_Model['Model Year'] <= 2023)]


In [19]:
fig = px.scatter(df_Model, x='Model Year', y = 'No_of_Vehicles', hover_name='Model',
                size = 'No_of_Vehicles', color='Model', log_x=True, size_max=60,
                title = 'Model with the highest number of vehicles each year')
fig.show()


The data indicates that while early electric vehicle adoption was led by models like the Nissan LEAF and Chevrolet Volt, the market has shifted significantly towards Tesla vehicles, particularly the Model Y in recent years.

The Tesla Model Y's surge in numbers, especially from 2021 onwards, reflects its strong market demand, likely driven by its versatility, range, and Tesla's reputation for innovation in the electric vehicle market.